In [1]:
import pandas as pd
import numpy as np
import sys
from sklearn.linear_model import LinearRegression
import import_ipynb
from hall_data import doping, anneal

importing Jupyter notebook from hall_data.ipynb


# Get image data

In [29]:
sample_datafile = '../data/rho_c/sample_images/data.pkl'
image_df = pd.read_pickle(sample_datafile)

Filter out TNO data

In [30]:
image_df = image_df[image_df.apply(lambda x: 'TNO' not in x['sample name'], axis = 1)]

Handle sample names

In [31]:
def extract_parameters(x):
    keys = ['substrate', 'doping', 'anneal']
    values = x['sample name'].split('_')
    res = dict(zip(keys, values))
    piece = 'piece' + str(int(x.piece))
    res['piece'] = piece
    return pd.Series({**x, **res})

image_df = image_df.apply(extract_parameters, axis = 1)

Calculate sample area.

In [43]:
COIN_RADIUS = 20.25 /(2*10) # cm (diameter to radius, mm to cm)
REFERENCE_AREA = np.pi*COIN_RADIUS**2 # cm^2

def get_area(x):
    return REFERENCE_AREA*x['pixels']/x['ref pixels']

image_df = image_df.assign(area = get_area)

# Get IVt data

In [32]:
IVt_folder = '../data/rho_c/'
sys.path.append(IVt_folder)
import getData
rho_df = getData.get_all_data()

Fit measured resistance.

In [33]:
def fit_resistance(x):
    data = x.data
    [[R]] = LinearRegression().fit(data.I.values.reshape(-1,1),
                              data.V.values.reshape(-1,1)).coef_
    return R

rho_df = rho_df.apply(lambda x: pd.Series({**x, 'R' : fit_resistance(x)}), axis = 1)

# Merge image and resistance data

In [46]:
full_df = rho_df.merge(image_df, on = ['substrate', 'doping', 'anneal', 'piece'])
full_df.doping = full_df.doping.astype(doping)
#full_df.anneal = full_df.doping.astype(anneal)
full_df = full_df.assign(rho_s = lambda x: x.R*x.area)
full_df = full_df.assign(rho_c = lambda x: x.rho_s/2)

In [47]:
df = full_df[['substrate', 'doping', 'anneal', 'R', 'area', 'rho_c']]
df 

,substrate,doping,anneal,R,area,rho_c
0,130nplus,r48,400C,0.116467,0.807406,0.047018
1,130nplus,r48,400C,0.130697,0.807406,0.052763
2,260nplus,r96,500C,0.180149,0.692040,0.062335
3,pSi,int,500C,21.422518,0.565777,6.060188
4,130nplus,r48,500C,0.110228,0.660188,0.036386
5,260nplus,r96,500C,0.208020,0.385471,0.040093
6,260nplus,r96,500C,0.201022,0.385471,0.038744
7,260nplus,r96,400C,0.063699,0.714584,0.022759
8,pSi,r96,500C,0.124755,0.523058,0.032627
9,pSi,r96,500C,0.124755,0.832926,0.051956


In [2]:
if __name__ == "__main__":
    import seaborn as sns
    p = sns.catplot(x = 'anneal', y = 'rho_c', hue = 'doping', kind = 'point', data = full_df[full_df.substrate == 'pSi'])
    p.set(yscale = 'log')

NameError: name 'full_df' is not defined